# Basic visualisation of rigids with Paraview

[ParaView](https://www.paraview.org/) is a powerfull and widely used tool for visualisation of scientific data.

It is possible to generate paraview visualisation files of LMGC90 computations. Here is presented a very basic use of paraview regarding these fields.

First let us generate a simple 2D case and generate the associated files. A case similar to *rigid/pre/DK_BoxJC* example previously demonstrated.

## Generation

In [ ]:
from pathlib import Path
import numpy as np

from pylmgc90 import pre

datbox = Path('./DATBOX')
datbox.mkdir(exist_ok=True)

# 2D
dim = 2

# containers
bodies = pre.avatars()
mats   = pre.materials()
mods   = pre.models()
svs    = pre.see_tables()
tacts  = pre.tact_behavs()

# model
mod = pre.model(name='rigid', physics='MECAx', element='Rxx2D', dimension=dim)
mods.addModel(mod)

# materials
tdur = pre.material(name='TDURx',materialType='RIGID',density=1000.)
plex = pre.material(name='PLEXx',materialType='RIGID',density=100.)
mats.addMaterial(tdur,plex)

# generating 1000 particles with random radius between [0.5, 2.[
nb_particles = 1000
radii = pre.granulo_Random(nb_particles, 0.5, 2.)
radius_min = min(radii)
radius_max = max(radii)

# deposit in a rectangular box
lx = 75.
ly = 50. 
nb_remaining_particles, coor, radii = pre.depositInBox2D(radii, lx, ly)

for r, c in zip(radii, coor):
    body = pre.rigidDisk(r=r, center=c, model=mod, material=plex, color='BLUEx') 
    bodies += body

# lower wall
down = pre.rigidJonc(axe1=0.5*lx+radius_max, axe2=radius_max, center=[0.5*lx, -radius_max],
                     model=mod, material=tdur, color='WALLx')
down.imposeDrivenDof(component=[1, 2, 3], dofty='vlocy')
bodies += down

# interactions management:
ldkdk = pre.tact_behav(name='iqsc0',law='IQS_CLB',fric=0.3)
tacts += ldkdk
#       - avec les parois
ldkjc = pre.tact_behav(name='iqsc1',law='IQS_CLB',fric=0.5)
tacts += ldkjc

svdkdk = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='DISKx', colorCandidat   ='BLUEx', behav=ldkdk,
                       CorpsAntagoniste='RBDY2', antagoniste='DISKx', colorAntagoniste='BLUEx', alert=0.1*radius_min)
svs+=svdkdk

svdkjc = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='DISKx', colorCandidat   ='BLUEx', behav=ldkjc,
                       CorpsAntagoniste='RBDY2', antagoniste='JONCx', colorAntagoniste='WALLx', alert=0.1*radius_min)
svs+=svdkjc

# orienting gravity
g = 9.81
angle = np.pi/4
gravity = np.array( [g*np.cos(angle), -g*np.sin(angle), 0.] )


post = pre.postpro_commands()
my_command=pre.postpro_command(name='SOLVER INFORMATIONS', step=1)
post.addCommand(my_command)

# input files writing
pre.writeDatbox(dim, mats, mods, bodies, tacts, svs, post=post, datbox_path=datbox)

#pre.visuAvatars(bodies)

## Computation

The only parameters of interest currently are the frequency at which the visualisation files will be generated. Usually this variable is named `freq_display`.

In [ ]:
from pylmgc90 import chipy
from pylmgc90.chipy import computation
import numpy as np

# space dimension
dim = 2

# time evolution parameters
dt = 1e-3
nb_steps = 300

# theta integrator parameter
theta = 0.5

# nlgs parameters
stype = 'Stored_Delassus_Loops       '
norm  = 'Quad ' 
conv  = 1e-4
relax = 1.0
gsit1 = 500
gsit2 = 10

# write parameter
freq_write   = 10

# display parameters
freq_display = 10

# x periodicity
lx = 75.

computation.initialize( dim, dt, theta, h5_file='lmgc90.h5' )
chipy.SetPeriodicCondition(xperiod=lx)

for k in range(1, nb_steps+1):
    if k%30 == 0:
        print( f"computing step {k}" )
    computation.one_step(stype, norm, conv, relax, gsit1, gsit2, freq_write, freq_display)

computation.finalize()

In [ ]:
#!ls DISPLAY/
#!paraview
#!/Applications/.../paraview

## Paraview first steps

First thing is to start paraview. The default view shoud be something like this:
![paraview_empty.png](./images/paraview_empty.png)
<br>
The second thing would be to open some files generated with LMGC90 by using *File/Open* menu button, the *ctrl+o* keyboard shortcut or clicking the icon on the left of the upper toolbar (where the pointer is on the above screenshot).

All display files generated by LMGC90 are stored in a *DISPLAY* folder next to the *DATBOX* one. When opening this folder with paraview the following window should appear:
![paraview_open.png](./images/paraview_open.png)
<br>

There are three lines, the first one with the directory name _lmgc90\_.._ can be ignored here (but not deleted though). The second one has the _pvd_ extension and the third one starting with the _+_ symbol and the _vtmb_ extension. In fact, for each recording, there is a single file to display the iteration number. The list of files can be unfolded by clicking the  _+_ symbol. Or all the files can be selected at once by clicking the folded line. But in this case, the matching between a file number and the simulation time is lost. This is were the _pvd_ file enters the fray: this file will associate a simulation time with one of the previous file. All this lengthy explaination to advise you to open the `lmgc90.pvd` file.


## Pipeline

If paraview configuration has not been tweaked, by default when reading the different files nothing is shown:
![paraview_pipeline.png](./images/paraview_pipeline.png)
<br>

That is because the data displayed with paraview follow the logic of a pipeline of operations on the raw data stored in the input files. As such, by default, the rendering of the view is done only when asked for. The idea is, in case of very large data, to not compute every rendering step of the visualisation pipeline, but only the desired ending step.

So to finally display something, the **Apply** green button must be pushed. There is an option in the *Edit/Settings* to check to automatically apply at every step.

The global arragement (which can partly be modified by the user) is:

* The upper line with menu and toolbars buttons
* The left part whith:
  * the pipeline browser on the upper part
  * the property browser (of the selected element in the pipeline) on the lower part
* The rendering zone (the rest).

In fact, all those browsers and toolbars panels can be moved around with the mouse and arranged in a different order.


## MultiBlocks

The *vtmb* extension stands for *vtk multi-block*. to access the different type of data represented in the file, the *MultiBlock inspector* must be activated if it is not already displayed yet. It can be done by clicking the **View** menu and ticking the the **MultiBlock inspector** box which will appear on the right side. Again this menu can be moved around and changed in size.

![paraview_multiblock.png](./images/paraview_multiblock.png)


For purely rigid computations, by default, there are three types of block that can be extracted:
* `ptc`: representing interaction.
* `rigids`: which contain data linked to the rigid bodies. There is no other geometrical information than the position of the center of inertia stored within this block.
* `tacts`: the main block to read when dealing with rigids since these files stores the geometrical information of the contactors.

In the multiblock inspector window, selecting only one of this block and then clicking the **Extract Blocks** button will create a new element in the pipeline browser. Again the **Apply** button in the pipeline inspector must be clicked to validate the extraction.

![paraview_extracting.png](./images/paraview_extracting.png)

For conveniencys' sake it is possible to click on this new entry in the pipeline browser to rename it. Then you can reproduce this step for all three blocks. Do not forget to click again on the `lmgc90.pvd` line in the pipeline browser before changing the block selection in the multiblock inspector, and to click the **Apply** button after the extraction to validate it on the pipeline browser side. In the end you should try to obtain something like this:

![paraview_extracted.png](./images/paraview_extracted.png)



## Basic tips

* In the pipeline browser, some elements can be hidden/display by clicking the *eye* icone on the left side of the name of the element of the pipeline.
* The main properties of use in this simple case is currently the opacity.
* In the render window, make sur that the interaction mode is 2D or 3D depending on your files (just under the layout tab)
    * In 2D mode
        * left click moves the scene
        * right click zooms in or out
        * middle click rotates the scene
    * In 3D mode
        * left click rotates the scene
        * right click zooms in or out
        * middle click moves the scene

If you have several files representing different time steps, you can view an a animation of your simulation by clicking the _play_ green icon on the first toolbar. You can also go to the first/last file or move only one step
forward or backward with all the surrounding buttons. The current time and step file are written in the next field.


## Fields and colors

When selecting the *tacts* in the pipeline browser, on the second toolbar, there is a dropdown menu allowing to chose what to display. Currently, for a 2D example, the different values are:

* `Disp`: the displacement vector
* `Ids`: the integer id of the contactor
* `Material`: the integer material id
* `Reac`: the contact reaction force vector apply to the rigid body
* `Rot Z`: the angle along z axis
* `Spin Z`: the angular velocity along z axis
* `Torque Z`: the moment due to contact force along z axis
* `Velocy`: the velocity vector

When selecting the *ptc* in the pipeline browser, the fields available are:

* `N`: the normal vector of the local contact frame
* `R`: the reaction vector in global frame
* `T`: the (first) tangential vector of the local contact frame
* `anbdy` : the model type id of the antagonist body
* `behav` : the id of the contact law
* `cdbdy` : the model type id of the candidate body
* `gap` : the gap
* `ianbdy` : the id of the antagonist body
* `icdan` : the id of the interaction (in its type numbering)
* `icdbdy` : the id of the candidate body
* `inter` : the interaction type id
* `rln`: the normal local reaction
* `rlt`: the tangential local reaction
* `status` : an integer id defining the contact state
* `vln`: the normal local velocity
* `vlt`: the tangential local velocity


In case of a vector field chosen, the next dropdown menu allow to chose to display a single component or the norm of the vector.

The next dropdown menu allow to change the apparence of the geometries displayed. Usually the interesting one for LMGC90 contactors are `Surface`, `Surface with edges` or `Wireframe`.

Finally, the color is managed with the five icons on the left of the toolbar. Allowing to reset the range, or to detect the range along the different time steps.

## Actions

Here are presented some of the basic actions that can be used to extract relevant informations from a scene view. Each actions described here applies only to the source it has been defined. So if a clipping action is needed it must be apply once to each source files (for example *tacts* and *ptc*).

Furthermore, it is possible to get back to the source, without having to remove the action result, just by activating or deactivating the view thanks to the eye icon next to each view.

### Clip

Allows to define a plan and render invisible everything on one side. Please note that, if the result of the clipping operation inherits the coloring (field, scale) from its source, they are still independent. It is possible the coloring may be different for the source and the result of the operation in the clipping (changing the component or field to display).

![paraview_clip.png](./images/paraview_clip.png)
<br>

### Threshold

This action allow to select only a subset of the particles or interactions depending on a criteria. Some random ideas would be to display:

* only interactions of a specific type
* only interactions with a negative or null gap
* only bodies of a range of material.

The threshold is always done on a scalar value! As such to filter using a vector field, it must be specified if applied to the norm, a single component, any component or all of them.

In the example below, two threshold have been consecutively used, one to select only the interactions with negative (or null gap), and among these only the ones with a normal reaction value above 100000N:
![paraview_threshold.png](./images/paraview_threshold.png)
<br>

In this image, to see the interactions, a very small value of *opacity* has been used on the *tacts* source and the
interactions appears as white dots.

### Glyph

The main operation use on the interaction is the use of glyphs.
The idea is to attach a geometric element depending on a scalar or vector field value.

The simplest idea is to represent a vector by an arrow. Furhtermore paraview allows to color the arrow depending on any other field.

In the next example, a *Line* glyph is used to on the *ptc* files to display the interactions as lines.
The line can be oriented along the *N* normal reaction direction, or the *R* reaction direction depending
on the needs of the user.

![paraview_glyph.png](./images/paraview_glyph.png)
<br>

A simple exercice left to the reader is to use an *arrow glyph* on the *rigids* files to show the contact reaction applied to each body and coloring the arrow with the moment along z axis:

*Note*: by default, the glyph is not applied to all elements of the files but to an uniform distribution.
The *Glyph Mode* dropdown menu of the *Glyph Mode* section must be checked and maybe desactivated. The idea is
that this reprenstation may be costly in terms of RAM and GPU usage.



## About interactions representation

Once the line glyph is added to the *ptc* files, it is recommended to use a specific filtering tool call *tubes* (to look for in the *Filters/search* or *Filters/alphabetical* menu), which will allow to orient a tube along a vector or another and to change the radius of the cylinder depending on one of the scalar value or the norm of a vector.

When varying the radius depending on a value, the radius of the mininum value is used and and a scale factor
can be set to define the radius of the max value.

![paraview_final.png](./images/paraview_final.png)

Because these operations of glyph/filter additions must be done each time a new file is opened, the code provides some macros for paraview. The user must click on the *Macros* menu and select *Import new macro...*.
Then navigating to the source directory of LMGC90, the file *src/addons/paraview/LMGC90_ptc.py* must be selected.
This will add a *LMGC90_ptc* button on the second toolbar of paraview which will automatically apply:
* the line glyph orienting along the reaction
* then the tube filter scaling the radius along `rln` value
* finally use a threshold to remove all interactions with the `noctc` status


## Disk, sphere, cylinder...

In our visualization, the disks, spheres and cylinder appear roughly discretized. It is important to note that, for historical reasons, the paraview files generated by LMGC90 do not use the efficient tools to generate nice looking objects, but internal representation.

This internal representation of discretized objects is done only for the visualization. Regarding the contact detection, force application points and moment involved, the real mathematical description of the geometry is used.


## To make this easier

A set of macro has been written to make the extraction of the blocks and the preset of the interactions with the tubes a little faster. The macros can be managed with the **Macros** menu button. The path to find is, from the sources of lmgc90 source directory : `src/addons/paraview`. Any of the `.py` file can be added. Each one can be loaded and will create an new button on the upper toolbar. Then you can click on the button to activate the different actions. The usefull ones here are:

* LMGC90.py: will extract all blocks and add a threshold to display on visible ones
* LMGC90_ptc.py: will create the line and tube glyp and a threshold to exclude the contact with `noctc` status